# Benchmark of Symmetry-Based Circuit Mapping Algorithm
---
### Basic Information
**Description:** This script benchmarks the runtime of a symmetry-based circuit mapping algorithm and compares its efficiency with MAPOMATIC, an existing quantum circuit remapping scheme. Additionally, the runtime for scoring circuit mappings is recorded as well, demonstrating the superior performance of our vectorized mapping scoring approach. Particularly, we consider a quantum hardware with an octagonal coupling graph for benchmarking.

In [1]:
# Load packages
from rustworkx import *
from rustworkx.visualization import mpl_draw

from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_dag

import networkx as nx
import rustworkx as rx
import rustworkx.generators
import numpy as np
import time

%run functions/functions_set_graph.ipynb # import graph generator
%run functions/functions_subgraph_matching.ipynb # import subgraph matching functions
%run functions/functions_circuit_mapping.ipynb # import circuit mapping functions
%run functions/functions_test_circuits.ipynb # import the predefined circuits
%run functions/functions_precompilation.ipynb # import precompiling functions

In [ ]:
# coupling graph type
graph_type = 'octagonal'

# set quantum circuit
circuit_width = 5
oracle = dj_oracle(circuit_width - 1)
input_circ = deutsch_jozsa_algorithm(oracle, circuit_width - 1) # input circuit

# define a reduced hardware coupling map for precompilation
reduced_coupling_graph, node_dict, node_dict_reverse = set_octagonal_graph(5, 5)
directional_edge_list = list(reduced_coupling_graph.edge_list())
bidirectional_edge_list = []
for edge in directional_edge_list:
    bidirectional_edge_list.append((edge[0], edge[1]))
    bidirectional_edge_list.append((edge[1], edge[0]))
reduced_coupling_map = CouplingMap(bidirectional_edge_list)
# reduced_coupling_map.draw()

# precompilation, find an initial mapping of the circuit onto the hardware
circ = precompile(circuit=input_circ, cmap=reduced_coupling_map)

In [ ]:
# define the qubit connectivity required by the circuit
circ_interaction_graph = circuit_interaction_graph(circ)
mpl_draw(circ_interaction_graph, with_labels=True)

In [ ]:
input_circ.draw(output='mpl')

In [ ]:
circ.draw(output='mpl')

In [ ]:
# hardware parameters
hardware_height_list = np.arange(10, 110, 5) # number of octagons in a column
# hardware_height_list = np.arange(20, 70, 10) # number of octagons in a column

fast_runtime_list = [] # runtime of SBCM
fast_scoring_runtime_list = [] # time consumption of vectorized circuit mapping evaluation
qubit_count_list = [] # number of qubits in the hardware

for hardware_height in hardware_height_list:
    hardware_width = hardware_height # number of qubits in a row

    # set quantum backend topology
    coupling_graph, node_dict, node_dict_reverse = set_octagonal_graph(hardware_width, hardware_height)
    num_qubits = coupling_graph.num_nodes()
    backend = QuantumBackendFast(num_qubits)
    qubit_count_list.append(num_qubits)
    backend.set_topology(coupling_graph)

    # set quantum backend error rates
    # set error rate distribution parameters for single-qubit gates
    sigma = 10
    amplitude = -0.01
    offset = 0.01
    backend.set_gate_types({'x': 1, 'h': 1, 'u3': 1, 'cx': 2, 'swap': 2})
    for gate_type in ['x', 'h', 'u3']:
        backend.set_multiple_gate_error(
            gate_type, 
            distribution='normal', 
            node_dict=node_dict, 
            graph_type=graph_type, 
            graph_size=[hardware_width, hardware_height], 
            sigma=sigma, 
            amplitude=amplitude, 
            offset=offset
        )

    # set error rate distribution parameters for two-qubit gates CX
    sigma = 10
    amplitude = -0.02
    offset = 0.02
    gate_type = 'cx'
    backend.set_multiple_gate_error(
        gate_type,
        distribution='normal', 
        node_dict=node_dict, 
        graph_type=graph_type, 
        graph_size=[hardware_width, hardware_height], 
        sigma=sigma, 
        amplitude=amplitude, 
        offset=offset
    )

    # set error rate distribution parameters for two-qubit gates SWAP
    sigma = 10
    amplitude = -0.06
    offset = 0.06
    gate_type = 'swap'
    backend.set_multiple_gate_error(
        gate_type,
        distribution='normal', 
        node_dict=node_dict, 
        graph_type=graph_type, 
        graph_size=[hardware_width, hardware_height], 
        sigma=sigma, 
        amplitude=amplitude, 
        offset=offset
    )

    # set error rate distribution parameters for readout
    sigma = 10
    amplitude = -0.05
    offset = 0.05
    backend.set_multiple_readout_error(
        distribution='normal', 
        node_dict=node_dict, 
        graph_type=graph_type, 
        graph_size=[hardware_width, hardware_height], 
        sigma=sigma, 
        amplitude=amplitude, 
        offset=offset
    )

    start_time = time.time() # start timing

    # search for all isomorphic circuit layout
    center_generating_set = generating_set([hardware_width, hardware_height], graph_type, node_dict_reverse)
    layouts = symmetry_based_subgraph_matching_optimized(backend.coupling_graph, circ_interaction_graph, node_dict, node_dict_reverse, center_generating_set, graph_type, [hardware_width, hardware_height])

    subgraph_matching_runtime = time.time() - start_time # runtime of subgraph matching

    # sort the candidate layouts in terms of their expected error rates
    results_SBCM = evaluate_layouts_fast(circ, layouts, backend)
    
    SBCM_runtime = time.time() - start_time # terminate timing
    fast_runtime_list.append(SBCM_runtime)
    
    fast_scoring_runtime = SBCM_runtime - subgraph_matching_runtime
    fast_scoring_runtime_list.append(fast_scoring_runtime)

    print(f'Qubits in the hardware: {num_qubits}; Qubits in the circuit: {circuit_width}; SBCM runtime: {SBCM_runtime} seconds')

In [ ]:
# hardware parameters
# hardware_height_list = np.arange(100, 110, 10) # number of octagons in a column

Mapomatic_runtime_list = []
scoring_runtime_list = [] # time consumption of circuit mapping evaluation

for hardware_height in hardware_height_list:
    hardware_width = hardware_height # number of qubits in a row

    # set quantum backend topology
    coupling_graph, node_dict, node_dict_reverse = set_octagonal_graph(hardware_width, hardware_height)
    num_qubits = coupling_graph.num_nodes()
    backend = QuantumBackendFast(num_qubits)
    backend.set_topology(coupling_graph)
    
    # set quantum backend error rates
    # set error rate distribution parameters for single-qubit gates
    sigma = 10
    amplitude = -0.01
    offset = 0.01
    backend.set_gate_types({'x': 1, 'h': 1, 'u3': 1, 'cx': 2, 'swap': 2})
    for gate_type in ['x', 'h', 'u3']:
        backend.set_multiple_gate_error(
            gate_type, 
            distribution='normal', 
            node_dict=node_dict, 
            graph_type=graph_type, 
            graph_size=[hardware_width, hardware_height], 
            sigma=sigma, 
            amplitude=amplitude, 
            offset=offset
        )

    # set error rate distribution parameters for two-qubit gates CX
    sigma = 10
    amplitude = -0.02
    offset = 0.02
    gate_type = 'cx'
    backend.set_multiple_gate_error(
        gate_type,
        distribution='normal', 
        node_dict=node_dict, 
        graph_type=graph_type, 
        graph_size=[hardware_width, hardware_height], 
        sigma=sigma, 
        amplitude=amplitude, 
        offset=offset
    )

    # set error rate distribution parameters for two-qubit gates SWAP
    sigma = 10
    amplitude = -0.06
    offset = 0.06
    gate_type = 'swap'
    backend.set_multiple_gate_error(
        gate_type,
        distribution='normal', 
        node_dict=node_dict, 
        graph_type=graph_type, 
        graph_size=[hardware_width, hardware_height], 
        sigma=sigma, 
        amplitude=amplitude, 
        offset=offset
    )

    # set error rate distribution parameters for readout
    sigma = 10
    amplitude = -0.05
    offset = 0.05
    backend.set_multiple_readout_error(
        distribution='normal', 
        node_dict=node_dict, 
        graph_type=graph_type, 
        graph_size=[hardware_width, hardware_height], 
        sigma=sigma, 
        amplitude=amplitude, 
        offset=offset
    )

    start_time = time.time() # start timing

    # search for all isomorphic circuit layout
    layouts = mapping_VF2pp(backend.coupling_graph, circ_interaction_graph, circ)

    subgraph_matching_runtime = time.time() - start_time # runtime of subgraph matching

    # sort the candidate layouts in terms of their expected error rates
    results_Mapomatic = evaluate_layouts(circ, layouts, backend)
    
    circuit_mapping_runtime = time.time() - start_time # terminate timing
    Mapomatic_runtime_list.append(circuit_mapping_runtime)

    scoring_runtime = circuit_mapping_runtime - subgraph_matching_runtime
    scoring_runtime_list.append(scoring_runtime)

    print(f'Qubits in the hardware: {num_qubits}; Qubits in the circuit: {circuit_width}; Mapomatic runtime: {circuit_mapping_runtime} seconds')

In [ ]:
# verify the consistency of the results of SBCM and MAPOMATIC
num_circuit_mapping_SBCM = len(results_SBCM[0])
num_circuit_mapping_Mapomatic = len(results_Mapomatic)
# if the numbers of circuit mappings produced by MAPOMATIC and SBCM are equal
flag_1 = (num_circuit_mapping_SBCM == num_circuit_mapping_Mapomatic)

best_circuit_mapping_score_SBCM = results_SBCM[0][0]
best_circuit_mapping_score_Mapomatic = results_Mapomatic[0][1]
# if the highest circuit mapping score is the same for MAPOMATIC and SBCM
flag_2 = (best_circuit_mapping_score_SBCM == best_circuit_mapping_score_Mapomatic)

best_circuit_mapping_Mapomatic = results_Mapomatic[0][0]
# index of the best circuit mapping found by MAPOMATIC in the list of circuit mappings found by SBCM
index_best_circuit_mapping_Mapoamtic = list(results_SBCM[1]).index(best_circuit_mapping_Mapomatic)
# if the best circuit mapping found by MAPOMATIC is also the best mapping found by SBCM
flag_3 = (results_SBCM[0][0] == results_SBCM[0][index_best_circuit_mapping_Mapoamtic])

if flag_1 and flag_2 and flag_3:
    print('verification passes')
else:
    print('Warning: verification fails')

In [9]:
# save results
np.save(f'data/circuit_mapping_{graph_type}.npy', [np.array(qubit_count_list), np.array(fast_runtime_list), np.array(Mapomatic_runtime_list), np.array(fast_scoring_runtime_list), np.array(scoring_runtime_list)])

# load results
# loaded_arrays = np.load('data/circuit_mapping.npy')
# _fast_runtime_list = loaded_arrays[0]
# _Mapomatic_runtime_list = loaded_arrays[1]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

# plotting parameters
linewidth = 2
markersize = 5
marker = 'o'
colors = ['orangered', 'purple']
labels = ['MAPOMATIC', 'SBCM']
plt.rcParams.update({'font.size': 10}) # font size
matplotlib.rc('font', family='Arial') # font style
fig_width = 8.6 # width of figure in cm
fig_height = 8 # height of figure in cm
figsize = (fig_width/2.54, fig_height/2.54)
dpi = 1200

# plot runtime versus number of qubits
x = qubit_count_list
y1 = Mapomatic_runtime_list
y2 = fast_runtime_list

plt.figure(figsize=figsize)
plt.plot(x, y1, label=labels[0], linewidth=linewidth, marker=marker, markersize=markersize, color=colors[0])
plt.plot(x, y2, label=labels[1], linewidth=linewidth, marker=marker, markersize=markersize, color=colors[1])

plt.title(f'Runtime for mapping a circuit')
plt.xlabel('number of qubits in the hardware')
plt.ylabel('runtime (seconds)')

plt.legend()
plt.tight_layout()

plt.savefig(f'figures/circuit_mapping_runtime_{graph_type}.png', dpi=dpi)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

# plotting parameters
linewidth = 2
markersize = 5
marker = 'o'
colors = ['orangered', 'purple']
labels = ['MAPOMATIC', 'SBCM']
plt.rcParams.update({'font.size': 10}) # font size
matplotlib.rc('font', family='Arial') # font style
fig_width = 8.6 # width of figure in cm
fig_height = 8 # height of figure in cm
figsize = (fig_width/2.54, fig_height/2.54)
dpi = 1200

# plot runtime versus number of qubits
x = qubit_count_list
y1 = scoring_runtime_list
y2 = fast_scoring_runtime_list

plt.figure(figsize=figsize)
plt.plot(x, y1, label=labels[0], linewidth=linewidth, marker=marker, markersize=markersize, color=colors[0])
plt.plot(x, y2, label=labels[1], linewidth=linewidth, marker=marker, markersize=markersize, color=colors[1])

plt.title(f'Runtime for evaluating circuit mappings')
plt.xlabel('number of qubits in the hardware')
plt.ylabel('runtime (seconds)')

plt.legend()
plt.tight_layout()

plt.savefig(f'figures/scoring_runtime_{graph_type}.png', dpi=dpi)
plt.show()